In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'/home/lucky/Documents/data_science_interview/Udemy/My_Projects/chicken_binary'

In [4]:
ls

artifacts/        dvc.yaml  params.yaml       setup.py     venv/
Chicken_project/  LICENSE   README.md         src/
config/           logs/     requirements.txt  template.py
data.zip          main.py   research/         templates/


# Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [22]:
@dataclass(frozen=True)
class ModelConfig:
    model_dir: Path
    model_path: Path
    img_size: list
    channels: int
    batch_size: int   # set batch size for training
    epochs: int   # number of all epochs in training
    patience: int   #number of epochs to wait to adjust lr if monitored value does not improve
    stop_patience: int   # number of epochs to wait before stopping training if monitored value does not improve
    threshold: float  # if train accuracy is < threshold adjust monitor accuracy, else monitor validation loss
    factor: float   # factor to reduce lr by
    ask_epoch: int   # number of epochs to run before asking if you want to halt training

# Configuration Manager

In [23]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH): 
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_config(self) -> ModelConfig:
        config = self.config.model
        params_config = self.params
        
        create_directories([config.model_dir])

        model_config = ModelConfig(
            model_dir=Path(config.model_dir),
            model_path=Path(config.model_path),
            img_size=params_config.img_size,
            channels=params_config.channels,
            batch_size=params_config.batch_size,   # set batch size for training
            epochs=params_config.epochs, # number of all epochs in training
            patience=params_config.patience,   #number of epochs to wait to adjust lr if monitored value does not improve
            stop_patience=params_config.stop_patience,   # number of epochs to wait before stopping training if monitored value does not improve
            threshold=params_config.threshold,  # if train accuracy is < threshold adjust monitor accuracy, else monitor validation loss
            factor=params_config.factor,  # factor to reduce lr by
            ask_epoch=params_config.ask_epoch   #
        )

        return model_config

In [14]:
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [15]:
PARAMS_FILE_PATH

PosixPath('params.yaml')

In [16]:
read_yaml(PARAMS_FILE_PATH).img_size

[2023-12-09 17:02:33,739: INFO: common: yaml file: params.yaml loaded successfully]


BoxList([224, 224])

In [25]:
ConfigurationManager().get_model_config()

[2023-12-09 17:10:27,737: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-09 17:10:27,739: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 17:10:27,740: INFO: common: created directory at: artifacts]
[2023-12-09 17:10:27,741: INFO: common: created directory at: artifacts/model]


ModelConfig(model_dir=PosixPath('artifacts/model'), model_path=PosixPath('artifacts/model/model_updated.h5'), img_size=BoxList([224, 224]), channels=3, batch_size=40, epochs=40, patience=1, stop_patience=3, threshold=0.9, factor=0.5, ask_epoch=5)

# Prepare Model

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax

In [27]:
class PrepareModel:
    def __init__(self, config: ModelConfig):
        self.model = None
        self.config = config


    def prepare_model(self):
        model = Sequential([
                Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]),
                BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
                MaxPooling2D(pool_size=2, strides=2),
                Conv2D(filters=32, kernel_size=3, activation='relu'),
                MaxPooling2D(pool_size=2, strides=2),
                Flatten(),
                Dense(units=128, activation='relu'),
                Dense(units=1, activation='sigmoid')
            ])
        model.compile(Adamax(learning_rate= 0.001), loss= 'binary_crossentropy', metrics= ['accuracy'])
        model.summary()
        self.model = model
        self.save_model(path=self.config.model_path, model=self.model)
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [28]:
try:
    config = ConfigurationManager()
    model_config = config.get_model_config()
    model_obj = PrepareModel(config=model_config)
    model_obj.prepare_model()
except Exception as e:
    raise e

[2023-12-09 17:10:38,997: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-09 17:10:39,000: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 17:10:39,001: INFO: common: created directory at: artifacts]
[2023-12-09 17:10:39,002: INFO: common: created directory at: artifacts/model]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization_1 (Bat  (None, 222, 222, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                         